In [ ]:
import os
from os.path import join as ojoin

sim_dir = ojoin('..', 'simulation_results') 

In [ ]:
def extract_coverage(fpath):
    total_coverage = 0
    with open(fpath) as f:
        for line in f:
            line = line.strip()
            if not line: continue
            total_coverage += float(line)
    return total_coverage

In [ ]:
best_coverage = [extract_coverage(ojoin(sim_dir, 'tim', 'output_%d.txt'%i)) for i in range(20)]

In [ ]:
pof_dict = dict()

for attribute in ('age', 'gender', 'ethnicity'):
    for approach in ('fairim', 'mip'):
        output_dir = ojoin(sim_dir, approach, 'group_maximin', attribute)
        pofs = []
        for graph_number in range(20):
            output_file = ojoin(output_dir, 'output_%d.txt'%graph_number)
            pof = best_coverage[graph_number] / extract_coverage(output_file)
            pofs.append(pof)
        pof_dict[(approach, attribute)] = pofs

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np


values = []
for attribute in ('gender', 'age', 'ethnicity'):
    for approach in ('fairim', 'mip'):
        values.append(pof_dict[(approach, attribute)])

means = [np.mean(v) for v in values]
stds = [np.std(v) for v in values]
xpos = []
for i in range(3):
    xpos.append(i)
    xpos.append(i + 0.35)

bars = plt.bar(xpos, means, yerr=stds, align='center', capsize=4, width=0.35)
colors = ['#606060', '#bababa'] * 3
for i in range(6):
    bars[i].set_color(colors[i])
    
labels = ('gender', 'age', 'ethnicity')
locs = [i + 0.17 for i in range(3)]
_ = plt.xticks(locs, labels, fontsize=13)

_ = plt.ylabel('Price of Fairness', fontsize=13)
_ = plt.legend(bars[:2], ('fairIM', 'MIP'), bbox_to_anchor=(1, 1.01), loc=2)
_ = plt.savefig('images/minimax_pof.pdf', bbox_inches='tight')